12/03/2021: Download Uniref100: Lugh 

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="uniref100"
#SBATCH -o uniref100_download.out
#SBATCH -p highmem
#SBATCH -n 16
wget ftp://ftp.ebi.ac.uk/pub/databases/uniprot/uniref/uniref100/uniref100.fasta.gz

[htrinh@lugh snf2]$ sbatch uniref100_download.sh

Install HMMer3

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmm3_download"
#SBATCH -o hmm3_download.out
#SBATCH -p normal
#SBATCH -n 8
set -eux
wget http://eddylab.org/software/hmmer/hmmer.tar.gz
tar xf hmmer.tar.gz
cd hmmer-3.3.2/
./configure --prefix=/data/htrinh/snf2/
make
make install

[htrinh@lugh snf2]$ sbatch hmm3_download.sh

15/03/2021: Extract uniref100.fasta.gz

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmm3_gunzip"
#SBATCH -o hmm3_gunzip.out
#SBATCH -p normal
#SBATCH -n 8
set -eux
gunzip uniref100.fasta.gz

Download MUSCLE (version 3.8.31): Align 1306 sequences, output is afa

In [ ]:
%%bash
wget https://www.drive5.com/muscle/downloads3.8.31/muscle3.8.31_i86linux64.tar.gz
tar xf muscle3.8.31_i86linux64.tar.gz

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="muscle_snf2"
#SBATCH -o muscle_snf2.out
#SBATCH -p normal
#SBATCH -n 8
./muscle3.8.31_i86linux64 -in snf2all.fa -out snf2all.afa -maxiters 1 -diags1 -sv -distance1 kbit20_3

Build a profile from the alignment (snf2all.afa)

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmmbuild_snf2afa"
#SBATCH -o hmmbuild_snf2afa.out
#SBATCH -p normal
#SBATCH -n 8
./hmmer-3.3.2/src/hmmbuild --informat afa snf2.hmm snf2all.afa

Extract the first 20 sequences of uniref100.fasta

In [ ]:
%%bash
awk "/^>/ {n++} n>20 {exit} {print}" uniref100.fasta > test_uniref100.fasta

Speed between hmmsearch and hmmscan: https://www.biostars.org/p/438243/
Speed of hmm search: https://www.biostars.org/p/179555/
hmmsearch: http://www.csb.yale.edu/userguides/seq/hmmer/docs/node26.html
Extracting HMMER results to sequence files: http://cryptogenomicon.org/extracting-hmmer-results-to-sequence-files-easel-miniapplications.html
Option for HMMer webtool: https://hmmer-web-docs.readthedocs.io/en/latest/searches.html

Searching a sequence database with a profile: search the sequence database with hmmsearch: Test with hmmsearch

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmmsearch_snf2"
#SBATCH -o hmmsearch_snf2.out
#SBATCH -p normal
#SBATCH -n 8
./hmmer-3.3.2/src/hmmsearch -A test_myhits.sto snf2.hmm test_uniref100.fasta > test_hmmsearch_snf2.out
./esl-reformat fasta ../../../test_myhits.sto > test_myhits.fa

Test with 6.8GB uniref100.fasta

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmmsearch_snf2_2"
#SBATCH -o hmmsearch_snf2_2.out
#SBATCH -p normal
#SBATCH -n 8
./hmmer-3.3.2/src/hmmsearch -Z 2000000 -A snf2_myhits_2.sto snf2.hmm test2_uniref100.fasta > hmmsearch_snf2_result_2.out

Run hmmsearch 

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmmsearch_snf2"
#SBATCH -o hmmsearch_snf2.out
#SBATCH -p highmem
#SBATCH -n 16
./hmmer-3.3.2/src/hmmsearch -Z 2000000 -A snf2_myhits.sto snf2.hmm uniref100.fasta > hmmsearch_snf2_result.out

16/03/2021: Extract ">" from test_myhits.fa

In [ ]:
%%bash
grep -o -E "^>\w+" test_myhits.fa | tr -d ">"|sort -u - > test_myhits.list

Index database fasta file to esl-sfetch

In [ ]:
%%bash
./hmmer-3.3.2/easel/miniapps/esl-sfetch --index test_uniref100.fasta

 Extract (sub)sequence(s) from a large sequence test_uniref100.fasta

In [ ]:
%%bash
./hmmer-3.3.2/easel/miniapps/esl-sfetch -f test_uniref100.fasta test_myhits.list > test_myhits_full.fa
less test_myhits_full.fa

>UniRef100_Q6GZW6 Putative helicase 009L n=4 Tax=Frog virus 3 TaxID=10493 RepID=009L_FRG3G
MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRRAVQDDPAFGHQKLV
ETFLSEDTPYRELLLFHAPGTGKTCTVVSVAERAKEKGLTRGCIVLARGAALLRNFLHEL
VFNCGTGGRYIPEGYADMGDQERTRKMRKAVSSYYQFRTYETFAKSVATMSAEAIRARYD
RFVIVMDEVHHLRSVQAEGVNTYSAISRFLRTVRGCVKMLLTGTPMTNEPGELADVLNLI
LPQDKTIRPEDGIFSNSGDLLKPDELAERVRGRVSYLKAARPDAGLTFAGEVLGGTGMTH
LRLVRLEMSAFQSDAYASAWDQDAGDRNIFSNSRQCSLAVMPDRRWGSAAEARNPSQVRR
MAGQNLAEYSVKYDYLVRVASSSPKTFAYCEYVNGSGLSLLSDILLANGWRRATGRETTP

Extract list ID from snf2_myhits.sto

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="snf2_ID"
#SBATCH -o snf2_ID.out
#SBATCH -p normal
#SBATCH -n 8
set -eux
./hmmer-3.3.2/easel/miniapps/esl-reformat fasta snf2_myhits.sto > snf2_myhits.fa
grep -o -E "^>\w+" snf2_myhits.fa | tr -d ">"|sort -u - > snf2_myhits.list

Index uniref100.fasta 

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="index_uniref100"
#SBATCH -o index_uniref100.out
#SBATCH -p highmem
#SBATCH -n 16
set -eux
./hmmer-3.3.2/easel/miniapps/esl-sfetch --index uniref100.fasta

How many snf2 sequences in the first draft: 426297

In [ ]:
%%bash
wc -l snf2_myhits.list

Extract full sequences from snf2_myhits.list to create a full list of first draft

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="first_draft_snf2"
#SBATCH -o first_draft_snf2.out
#SBATCH -p highmem
#SBATCH -n 16
./hmmer-3.3.2/easel/miniapps/esl-sfetch -f uniref100.fasta snf2_myhits.list > snf2_myhits_full.fa

18/03/2021:  hmmalign --trim 

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="hmmalign_snf2"
#SBATCH -o hmmalign_snf2.out
#SBATCH -p MSC
#SBATCH --nodelist=compute01
#SBATCH --partition=MSC
#SBATCH -n 8
./hmmer-3.3.2/src/hmmalign --trim snf2.hmm snf2_myhits_full.fa

In [ ]:
%%bash
mv hmmalign_snf2.out hmmalign_snf2.sto

In [ ]:
%%bash
#!/bin/bash
#SBATCH --job-name="snf2_helicase_regions"
#SBATCH -o snf2_helicase_regions.out
#SBATCH -p highmem
#SBATCH --nodelist=compute06,compute07,compute08
#SBATCH -n 16
set -eux
./hmmer-3.3.2/easel/miniapps/esl-reformat fasta hmmalign_snf2.sto > hmmalign_snf2.fa

23/03/2021: Remove unimportant files

In [ ]:
%%bash
rm test2_uniref100.fasta

Rerun with E-value 0.001

In [ ]:
%%bash
vim hmmsearch_snf2.sh
#!/bin/bash
#SBATCH --job-name="hmmsearch_snf2_E-value"
#SBATCH -o hmmsearch_snf2.out
#SBATCH -p highmem
#SBATCH --nodelist=compute06,compute07,compute08
#SBATCH -n 40
./hmmer-3.3.2/src/hmmsearch -Z 2000000 --tblout hmmsearch_snf2_result_e_value.out -E 1e-03 snf2.hmm uniref100.fasta 

24/03/2020: Download 2 files from Andrew Flaus:
hmmsearch-evalues-E-75-uIDonly
hmmsearch-evalues-E-120-uIDonly

In [ ]:
%%bash
vim andrewE-75.sh
#!/bin/bash
#SBATCH --job-name="andrewE-75"
#SBATCH -o andrewE-75.out
#SBATCH -p highmem
#SBATCH --nodelist=compute06
#SBATCH -n 16
./hmmer-3.3.2/easel/miniapps/esl-sfetch -f uniref100.fasta hmmsearch-evalues-E-75-uIDonly.txt > hmmSnf2-E-75.sto
./hmmer-3.3.2/src/hmmalign --trim snf2.hmm hmmSnf2-E-75.sto > hmmSnf2-E-75-trim.sto
./hmmer-3.3.2/easel/miniapps/esl-reformat afa hmmSnf2-E-75-trim.sto > hmmSnf2-E-75-trim.fa

In [ ]:
#!/bin/bash
#SBATCH --job-name="andrewE-120"
#SBATCH -o andrewE-120.out
#SBATCH -p highmem
#SBATCH --nodelist=compute06
#SBATCH -n 16
./hmmer-3.3.2/easel/miniapps/esl-sfetch -f uniref100.fasta hmmsearch-evalues-E-120-uIDonly.txt > hmmSnf2-E-120.sto
./hmmer-3.3.2/src/hmmalign --trim snf2.hmm hmmSnf2-E-120.sto > hmmSnf2-E-120-trim.sto
./hmmer-3.3.2/easel/miniapps/esl-reformat afa hmmSnf2-E-120-trim.sto > hmmSnf2-E-120-trim.fa

hmmSnf2-E-120-trim.fa is afa alignment file

25/03/2021: Download the old hmm file from Andrew: snf2all.hmm

Run hmmsearch on the 1306 sequences only using your new snf2.hmm

In [ ]:
#!/bin/bash
#SBATCH --job-name=""
#SBATCH -o hmmsearch_snf2.out
#SBATCH -p highmem
#SBATCH --nodelist=compute06,compute07,compute08
#SBATCH -n 40
./hmmer-3.3.2/src/hmmsearch -Z 2000000 --tblout hmmsearch_snf2_result_e_value.out -E 1e-03 snf2.hmm uniref100.fasta